In [48]:
# VER TODO-LIST em README.md
import json
import urllib.request
from toolset import (get_examples, get_frequency,
                    get_text, get_tokens)

from tools import (get_word_data, get_word_fallback, 
                   produce_dataframe, append_to_csv)
import nltk

import re

dream, wf = get_text("dream.txt")

In [6]:
import spacy
nlp = spacy.load("ru_core_news_lg")
doc = nlp(dream)

In [135]:
tokens, token_set, phrases = get_tokens(dream)
frequency = get_frequency(tokens)

wordlist = []
for pair in frequency:
    token = pair[0]
    wordlist.append(token)

In [137]:
tokens = [str(token) for token in doc if not (token.is_punct or token.is_quote or token.is_space or token.is_bracket)]
lemmas = [token.lemma_ for token in doc if not (token.is_punct or token.is_quote or token.is_space or token.is_bracket)]
type(doc[0])

spacy.tokens.token.Token

In [46]:
def request(action, **params):
    return {'action': action,
            'params': params,
            'version': 6}

def invoke(action, **params):
    requestJson = json.dumps(request(action, **params)).encode('utf-8')
    response = json.load(urllib.request.urlopen(urllib.request.Request('http://localhost:8765', requestJson)))

    if len(response) != 2:
        raise Exception('response has an unexpected number of fields')

    if 'error' not in response:
        raise Exception('response is missing required error field')
    
    if 'result' not in response:
        raise Exception('response is missing required result field')

    if response['error'] is not None:
        raise Exception(response['error'])

    return response['result']

noteIDs = invoke('findNotes', query='deck:Línguas::Русский')
notesInfo = invoke('notesInfo', notes=noteIDs)

In [159]:
len(invoke('findNotes', query='deck:Línguas::Русский::tri-rasskazy'))

398

In [145]:
notes = []
for card in notesInfo:
    notes.append(card['fields']['Frente']['value'])

def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)

def striptags(data):
    p = re.compile(r'\[.*?\]')
    return p.sub('', data)

def stripparens(data):
    p = re.compile(r'\(.*?\)')context_api
    return p.sub('', data)

notes = [re.split(' |_|-|!|\+\&;', note)[0] for note in notes]
notes = [re.split('\(', note)[0] for note in notes]
notes = [re.split('\[', note)[0] for note in notes]
notes = [re.split('<', note)[0] for note in notes]
notes = [re.split('\n', note)[0] for note in notes]
notes = [re.split('$', note)[0] for note in notes]
notes = [re.split('&', note)[0] for note in notes]
notes = [stripparens(note) for note in notes]
notes = [striphtml(note) for note in notes]
notes = [striptags(note) for note in notes]

notes = list(filter(None, notes))
notes_tokenized = nlp(spacy.tokens.Doc(nlp.vocab,notes))

In [163]:
notes_tokens = [str(token) for token in notes_tokenized if not (token.is_punct or token.is_quote or token.is_bracket or token.is_space)]
notes_lemmas = [token.lemma_ for token in notes_tokenized if not (token.is_punct or token.is_quote or token.is_bracket or token.is_space)]

dream_content = zip(tokens, lemmas)
cards_content = zip(notes_tokens, notes_lemmas)


acceptable = []

for pair in dream_content:
    if (pair[1] not in notes_lemmas) and (pair[0] not in notes_tokens):
        acceptable.append(pair[0])

acceptable = set(acceptable)

{'Скорби',
 'Надо',
 'абсолютный',
 'ужилась',
 'упрека',
 'ознобе',
 'ворчали',
 'одном',
 'крайней',
 'готовый',
 'чине',
 'сказывалась',
 'Мамочка',
 'прочувствовал',
 'ужаснулись',
 'восполнившегося',
 'сновидения',
 'стремятся',
 'натыкаюсь',
 'пуля',
 'отчаянии',
 'дорогами',
 'воды',
 'него',
 'глазах',
 'договаривала',
 'восполнившимся',
 'проникновениями',
 'согласном',
 'крестятся',
 'умножившаяся',
 'самым',
 'веруя',
 'чтоб',
 'равно',
 'славили',
 'оставался',
 'ни',
 'отрывисто',
 'видывал',
 'оставалось',
 'кончилось',
 'полагал',
 'различить',
 'отозвалась',
 'невинными',
 'восполнены',
 'иль',
 'спутнику',
 'мной',
 'невинное',
 'скрыть',
 'неразрешимым',
 'живет',
 'нынешнюю',
 'двойник',
 'перегородкой',
 'иными',
 'ночь',
 'сержусь',
 'страдать',
 'борьба',
 'утратили',
 'добивались',
 'спокойны',
 'захотят',
 'грезит',
 'атома',
 'прохожий',
 'подобными',
 'участникам',
 'ним',
 'часу',
 'ими',
 'направит',
 'веровать',
 'книги',
 'Ох',
 'торжеством',
 'побежденные

In [3]:
import pandas as pd

#df = pd.DataFrame(acceptable)
#df.to_csv("output.csv", encoding="utf-8", header=None)

In [4]:
new_df = pd.read_csv("filtered-ru.csv", header=None, index_col=0)
new_df

""
0
архипелагу
Барин
басит
башмаки
безгрешную
...
Явились
явились
Явилось


In [18]:

clist = wf.concordance_list("человек")

In [30]:
clist[0][6]

'Я смешной человек . Они меня называют теперь сумасшед'

In [61]:
n = 0
for item in new_df.iterrows():
    n+=1
    concordance = wf.concordance_list(item[0])[0][6]
    cleaned_up = re.sub(r'\s([?.!,:;"](?:\s|$))', r'\1', concordance)
    print("{}:".format(item[0]), cleaned_up)
    if n >= 100:
        break

архипелагу: е материка, прилегающего к этому архипелагу. О, все было точно так же, как
Барин:  и крикнул. Она прокричала лишь: « Барин, барин! .. » но вдруг бросила меня
басит: о движения. Кругом ходят и кричат, басит капитан, визжит хозяйка, и вдруг о
башмаки: мнил особенно ее мокрые разорванные башмаки и теперь помню. Они мне особенно м
безгрешную: аразил собой всю эту счастливую, безгрешную до меня землю. Они научились лга
бездонные:  разорванные облака, а между ними бездонные черные пятна. Вдруг я заметил в о
безобразием:  за неразумное самоубийство мое — безобразием и нелепостью дальнейшего бытия, 
безотчетно: имо, были в ней до того убеждены безотчетно, что это не составляло для них в
берега: вое изумрудное море тихо плескало о берега и лобызало их с любовью, явной, в
бесился: аверно, но они горячили меня, и я бесился. Я как бы уже не мог умереть тепер
беспредельно: одна капля упала, но я знал, я беспредельно и нерушимо знал и верил, что не
беспрерывное: было какое-то насущное, ж

NameError: name 'concordance_list' is not defined